In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier

# Load waveform data (assumed to be in time-series format)
df = pd.read_csv("ecg_waveforms.csv")

# Extract only ECG leads (skip time column)
lead_columns = [
    'MDC_ECG_LEAD_I', 'MDC_ECG_LEAD_II', 'MDC_ECG_LEAD_V1', 'MDC_ECG_LEAD_V2',
    'MDC_ECG_LEAD_V3', 'MDC_ECG_LEAD_V4', 'MDC_ECG_LEAD_V5', 'MDC_ECG_LEAD_V6',
    'MDC_ECG_LEAD_III', 'MDC_ECG_LEAD_AVR', 'MDC_ECG_LEAD_AVL', 'MDC_ECG_LEAD_AVF'
]

# Extract features: mean, std, peak-to-peak range, and signal energy for each lead
feature_list = []
for lead in lead_columns:
    signal = df[lead].dropna().values
    feature_list.extend([
        np.mean(signal),
        np.std(signal),
        np.max(signal) - np.min(signal),
        np.sum(np.square(signal))  # energy
    ])

# Reshape into 2D array for model input
X = np.array(feature_list).reshape(1, -1)

# Define CardioForest model exactly as given
cardioforest = RandomForestClassifier(
    n_estimators=1000,
    max_depth=20,
    min_samples_split=5,
    min_samples_leaf=2,
    max_features=0.6,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1,
    bootstrap=True,
    oob_score=True,
    ccp_alpha=0.01
)

# Predict WCT (Wide Complex Tachycardia)
prediction = cardioforest.predict(X)[0]
label = "WCT" if prediction == 1 else "Normal"

# Output results
print(f"Prediction: {label}")

# Save results
results_df = pd.DataFrame({
    'Prediction': [label],
    'Code': [int(prediction)]
})
results_df.to_csv("wct_prediction_result.csv", index=False)

# Pie chart for visualization
plt.figure(figsize=(6, 6), dpi=300)
plt.pie(
    [1],
    labels=[label],
    colors=['salmon' if label == 'WCT' else 'skyblue'],
    autopct='%1.1f%%',
    startangle=90,
    textprops={'fontsize': 14, 'fontname': 'Times New Roman'}
)
plt.title("WCT Prediction Result", fontsize=16, fontname='Times New Roman')
plt.tight_layout()
plt.savefig("wct_single_prediction_pie_chart.png")
plt.show()


NotFittedError: This RandomForestClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.